# Imports

In [1]:
from newsapi import NewsApiClient
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
import json
from bs4 import BeautifulSoup

# Source level categorization for countru and category

In [2]:
newsapi = NewsApiClient(api_key='3eb3d1ecdf60403f97e0352fcbc286b4')
data = newsapi.get_sources()

In [3]:
# Extracting information
status = data['status']
sources = data['sources']

In [4]:
# Listing sources 
source_list = []

for source in sources:
    id = source['id']
    name = source['name']
    source_list.append((name,id))
    
source_list

[('ABC News', 'abc-news'),
 ('ABC News (AU)', 'abc-news-au'),
 ('Aftenposten', 'aftenposten'),
 ('Al Jazeera English', 'al-jazeera-english'),
 ('ANSA.it', 'ansa'),
 ('Argaam', 'argaam'),
 ('Ars Technica', 'ars-technica'),
 ('Ary News', 'ary-news'),
 ('Associated Press', 'associated-press'),
 ('Australian Financial Review', 'australian-financial-review'),
 ('Axios', 'axios'),
 ('BBC News', 'bbc-news'),
 ('BBC Sport', 'bbc-sport'),
 ('Bild', 'bild'),
 ('Blasting News (BR)', 'blasting-news-br'),
 ('Bleacher Report', 'bleacher-report'),
 ('Bloomberg', 'bloomberg'),
 ('Breitbart News', 'breitbart-news'),
 ('Business Insider', 'business-insider'),
 ('Business Insider (UK)', 'business-insider-uk'),
 ('Buzzfeed', 'buzzfeed'),
 ('CBC News', 'cbc-news'),
 ('CBS News', 'cbs-news'),
 ('CNN', 'cnn'),
 ('CNN Spanish', 'cnn-es'),
 ('Crypto Coins News', 'crypto-coins-news'),
 ('Der Tagesspiegel', 'der-tagesspiegel'),
 ('Die Zeit', 'die-zeit'),
 ('El Mundo', 'el-mundo'),
 ('Engadget', 'engadget'),
 ('E

In [5]:
# # Filtering by categories
# categorie_source = {}
# for source in sources:
#     category = source['category']
#     if category not in categorie_source:
#         categorie_source[category] = []
#     categorie_source[category].append(source['name'])

# To be filtered manually and generate a mask

In [6]:
# Classify sources by category
categories = {}
countries = {}

for source in sources:
    category = source['category']
    country = source['country']
    
    if category not in categories:
        categories[category] = []
    categories[category].append(source)
    
    if country not in countries:
        countries[country] = []
    countries[country].append(source)

In [7]:
def create_dataframe(data_dict, outer_key_name, inner_key_name):
    records = []
    for outer_key, sources in data_dict.items():
        for source in sources:
            record = {
                outer_key_name: outer_key,
                inner_key_name: source['name'],
                'id': source['id'],
                'description': source['description'],
                'url': source['url'],
                'category': source['category'],
                'language': source['language'],
                'country': source['country']
            }
            records.append(record)
    return pd.DataFrame(records).set_index([outer_key_name, inner_key_name])

# Create DataFrame for categories
df_categories = create_dataframe(categories, 'Category', 'Source Name')
yesterday_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
df_categories['yesterday_date'] = yesterday_date
df_categories.drop(labels='category', axis=1, inplace=True)

# Create DataFrame for countries
df_countries = create_dataframe(countries, 'Country', 'Source Name')

Filtering mask will be applied

In [8]:
df_categories

id  \
Category      Source Name                                
general       ABC News                        abc-news   
              ABC News (AU)                abc-news-au   
              Aftenposten                  aftenposten   
              Al Jazeera English    al-jazeera-english   
              ANSA.it                             ansa   
...                                                ...   
entertainment The Lad Bible              the-lad-bible   
health        Medical News Today    medical-news-today   
science       National Geographic  national-geographic   
              New Scientist              new-scientist   
              Next Big Future          next-big-future   

                                                                         description  \
Category      Source Name                                                              
general       ABC News             Your trusted source for breaking news, analysi...   
              ABC News (AU)        Australia's most trusted source of local, nati...   
              Aftenposten          Norges ledende nettavis med alltid oppdaterte ...   
              Al Jazeera English   News, analysis from the Middle East and worldw...   
              ANSA.it              Agenzia ANSA: ultime notizie, foto, video e ap...   
...                                                                              ...   
entertainment The Lad Bible        The LAD Bible is one of the largest community ...   
health        Medical News Today   Medical news and health news headlines posted ...   
science       National Geographic  Reporting our world daily: original nature and...   
              New Scientist        Breaking science and technology news from arou...   
              Next Big Future      Coverage of science and technology that have t...   

                                                                         url  \
Category      Source Name                                                      
general       ABC News                                https://abcnews.go.com   
              ABC News (AU)                      https://www.abc.net.au/news   
              Aftenposten                         https://www.aftenposten.no   
              Al Jazeera English                   https://www.aljazeera.com   
              ANSA.it                                    https://www.ansa.it   
...                                                                      ...   
entertainment The Lad Bible                      https://www.theladbible.com   
health        Medical News Today             http://www.medicalnewstoday.com   
science       National Geographic         http://news.nationalgeographic.com   
              New Scientist        https://www.newscientist.com/section/news   
              Next Big Future                  https://www.nextbigfuture.com   

                                  language country yesterday_date  
Category      Source Name                                          
general       ABC News                  en      us     2024-06-07  
              ABC News (AU)             en      au     2024-06-07  
              Aftenposten               no      no     2024-06-07  
              Al Jazeera English        en      us     2024-06-07  
              ANSA.it                   it      it     2024-06-07  
...                                    ...     ...            ...  
entertainment The Lad Bible             en      gb     2024-06-07  
health        Medical News Today        en      us     2024-06-07  
science       National Geographic       en      us     2024-06-07  
              New Scientist             en      us     2024-06-07  
              Next Big Future           en      us     2024-06-07  

[128 rows x 6 columns]

In [9]:
# Define a function to fetch and convert JSON data to a DataFrame
def fetch_json(source_id, date):
    url = (f'https://newsapi.org/v2/everything?'
           f'sources={source_id}&'
           f'from={date}&'
           'sortBy=popularity&'
           'apiKey=3eb3d1ecdf60403f97e0352fcbc286b4')
    
    response = requests.get(url)
    return response.json()

In [29]:
def json_to_df(data):
    articles = data['articles']
    df = pd.DataFrame(articles)
    df = df[['author', 'title', 'url', 'description', 'publishedAt']]
    df.columns = ['author', 'title', 'url', 'description', 'date']
    
    # Adding the content column by scraping the URLs
    contents = []
    for url in df['url']:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            article_content = soup.find_all('p')
            content_text = ' '.join([paragraph.get_text() for paragraph in article_content])
            contents.append(content_text)
        except Exception as e:
            contents.append(f"Error: {str(e)}")
    
    df['content'] = contents
    return df

Run the next block only after filtering

In [30]:
# df_categories['data'] = df_categories.apply(
#     lambda row: fetch_and_convert_to_dataframe(row['id'], row['yesterday_date']),
#     axis=1
# )

demo

In [31]:
df_first_two_rows = df_categories.head(2).copy()
df_first_two_rows['data'] = df_first_two_rows.apply(
    lambda row: json_to_df(fetch_json(row['id'], row['yesterday_date'])),
    axis=1
)

In [32]:
df_first_two_rows['data'][0]['content'][6]

"Prince Harry has been given permission to appeal the British government's rejection to provide him with police protection in the U.K. The Court of Appeal gave the Duke of Sussex the go-ahead to challenge a ruling earlier this year in the High Court LONDON -- Prince Harry has been given permission to appeal the British government’s rejection to provide him with publicly funded police protection in the U.K.  The Court of Appeal gave the Duke of Sussex the go-ahead to challenge a ruling earlier this year in the High Court. The permission was granted in May but only reported Thursday. Judge Peter Lane ruled in February that a government panel’s decision to provide “bespoke” security on an as-needed basis after Harry quit as a working member of the royal family was not unlawful, irrational or unjustified.  “Insofar as the case-by-case approach may otherwise have caused difficulties, they have not been shown to be such as to overcome the high hurdle so as to render the decision-making irrat

In [ ]:
df_categories

id  \
Category      Source Name                                
general       ABC News                        abc-news   
              ABC News (AU)                abc-news-au   
              Aftenposten                  aftenposten   
              Al Jazeera English    al-jazeera-english   
              ANSA.it                             ansa   
...                                                ...   
entertainment The Lad Bible              the-lad-bible   
health        Medical News Today    medical-news-today   
science       National Geographic  national-geographic   
              New Scientist              new-scientist   
              Next Big Future          next-big-future   

                                                                         description  \
Category      Source Name                                                              
general       ABC News             Your trusted source for breaking news, analysi...   
              ABC News (AU)        Australia's most trusted source of local, nati...   
              Aftenposten          Norges ledende nettavis med alltid oppdaterte ...   
              Al Jazeera English   News, analysis from the Middle East and worldw...   
              ANSA.it              Agenzia ANSA: ultime notizie, foto, video e ap...   
...                                                                              ...   
entertainment The Lad Bible        The LAD Bible is one of the largest community ...   
health        Medical News Today   Medical news and health news headlines posted ...   
science       National Geographic  Reporting our world daily: original nature and...   
              New Scientist        Breaking science and technology news from arou...   
              Next Big Future      Coverage of science and technology that have t...   

                                                                         url  \
Category      Source Name                                                      
general       ABC News                                https://abcnews.go.com   
              ABC News (AU)                      https://www.abc.net.au/news   
              Aftenposten                         https://www.aftenposten.no   
              Al Jazeera English                   https://www.aljazeera.com   
              ANSA.it                                    https://www.ansa.it   
...                                                                      ...   
entertainment The Lad Bible                      https://www.theladbible.com   
health        Medical News Today             http://www.medicalnewstoday.com   
science       National Geographic         http://news.nationalgeographic.com   
              New Scientist        https://www.newscientist.com/section/news   
              Next Big Future                  https://www.nextbigfuture.com   

                                  language country yesterday_date  
Category      Source Name                                          
general       ABC News                  en      us     2024-06-07  
              ABC News (AU)             en      au     2024-06-07  
              Aftenposten               no      no     2024-06-07  
              Al Jazeera English        en      us     2024-06-07  
              ANSA.it                   it      it     2024-06-07  
...                                    ...     ...            ...  
entertainment The Lad Bible             en      gb     2024-06-07  
health        Medical News Today        en      us     2024-06-07  
science       National Geographic       en      us     2024-06-07  
              New Scientist             en      us     2024-06-07  
              Next Big Future           en      us     2024-06-07  

[128 rows x 6 columns]

# Hence pipeline created
will be improved later